<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2022-03-20 07:01:34--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  26.8MB/s    in 53s     

2022-03-20 07:02:29 (26.3 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` в топ-5 близких слов к слову `dog`? Какое место? 

**Ответ:** Слово `cats` входит в топ-5 близких слов и занимает 4ое место

In [ ]:
top_words = wv_embeddings.similar_by_word(word=word, topn=5)
top_words

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
import nltk
from nltk.tokenize import WordPunctTokenizer, word_tokenize

nltk.download('punkt')

class TokenizerInterface:
    def __repr__(self):
        return "TokenizerInterface"

    def tokenize(self, text):
        raise NotImplementedError

class MyTokenizer(TokenizerInterface):
    def __init__(self):
        pass

    def __repr__(self):
        return "MyTokenizer"

    def tokenize(self, text):
        return re.findall('\w+', text)

class WordTokenizer(TokenizerInterface):
    def __init__(self):
        pass

    def __repr__(self):
        return "WordTokenizer"

    def tokenize(self, text):
        return word_tokenize(text)

class NLTKTokenizer(TokenizerInterface):
    def __init__(self):
        self.tokenizer = WordPunctTokenizer()

    def __repr__(self):
        return "NLTKTokenizer"

    def tokenize(self, text):
        return self.tokenizer.tokenize(text)

tokenizer = MyTokenizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def question_to_vec(question, embeddings, tokenizer):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    def get_vector(word):
        try:
            word_emb = embeddings.get_vector(word)
            return word_emb

        except KeyError:
            return None
    
    tokenized_question = tokenizer.tokenize(question)

    emb_question = list(map(get_vector, tokenized_question))
    emb_question = list(filter(
        lambda elem: not elem is None, 
        emb_question
    ))
    emb_question = np.array(emb_question)

    return np.zeros(embeddings.vector_size) if emb_question.size == 0 \
           else emb_question.mean(axis=0)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

**Ответ:** -1.29

In [ ]:
question = "I love neural networks"
vec_question = question_to_vec(question, wv_embeddings, tokenizer)
round(vec_question[2], 2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

**Ответ:** 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

**Ответ:** $\dfrac{1}{log_2 (1 + 9)} \cdot [9 \leqslant 10] \approx 0.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    dup_ranks = np.array(dup_ranks)
    assert dup_ranks.dtype == int

    return np.mean(dup_ranks <= k)

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dup_ranks = np.array(dup_ranks)
    assert dup_ranks.dtype == int and np.all(dup_ranks >= 1)

    bool_mask = dup_ranks <= k
    coefs = 1 / np.log2(1 + dup_ranks)

    return np.mean(np.multiply(bool_mask, coefs))

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('stackoverflow_similar_questions.zip')

In [ ]:
!unzip -q stackoverflow_similar_questions.zip

Считайте данные.

In [ ]:
def read_corpus(filename, is_lower=False, norm_func=None):
    data = []
    for line in open(filename, encoding='utf-8'):
        if is_lower:
            line = line.lower()

        if not norm_func is None:
            line = norm_func(line)
        
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [ ]:
# Поскольку в предобученных эмбеддингах текст приведён к нижнему регистру, 
# то с валидационными данными поступим также.
validation_data = read_corpus('./data/validation.tsv', is_lower=True)

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_emb = question_to_vec(question, embeddings, tokenizer)
    candidates_emb = list(map(
        lambda string: question_to_vec(string, embeddings, tokenizer), 
        candidates
    ))
    candidates_emb = np.array(candidates_emb)

    cos_distances = cosine_similarity(X=candidates_emb, Y=question_emb[None, :])
    cos_distances = np.ravel(cos_distances)
    sorted_cand_inds = cos_distances.argsort()[::-1]

    return list(zip(sorted_cand_inds, np.array(candidates)[sorted_cand_inds]))

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
    print(ranks)
    print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(-1, 'Getting all list items of an unordered list in PHP'), #скрыт
            (-1, 'select2 not displaying search results'), #скрыт
            (-1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ:** 102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.410 | Hits@   1: 0.410
DCG@   5: 0.500 | Hits@   5: 0.580
DCG@  10: 0.521 | Hits@  10: 0.644
DCG@ 100: 0.568 | Hits@ 100: 0.875
DCG@ 500: 0.580 | Hits@ 500: 0.973
DCG@1000: 0.583 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv', is_lower=True)

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
def get_train_words(train_data, tokenizer):
    train_data = list(map(lambda lst: " ".join(lst), train_data))
    train_words = list(map(lambda line: tokenizer.tokenize(line), train_data))

    return train_words

In [ ]:
from gensim.models import Word2Vec


tokenizer = MyTokenizer()
train_words = get_train_words(train_data, tokenizer)
embeddings_trained = Word2Vec(train_words,
                              size=200,
                              min_count=5,
                              window=3).wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.290 | Hits@   1: 0.290
DCG@   5: 0.359 | Hits@   5: 0.420
DCG@  10: 0.378 | Hits@  10: 0.480
DCG@ 100: 0.435 | Hits@ 100: 0.760
DCG@ 500: 0.458 | Hits@ 500: 0.941
DCG@1000: 0.464 | Hits@1000: 1.000


**Наблюдение:** Качество на обученных эмбеддингах получилось заметно хуже, чем на предобученных, ибо не было произведена фильтрация слов от стоп-слов.

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

### Сравнение различных способов токенизации 

In [ ]:
def validate(validation_data, embeddings, tokenizer, max_val_examples=1000):
    wv_ranking = []

    for i, line in enumerate(tqdm(validation_data)):
        if i == max_val_examples:
            break

        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)

    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

In [ ]:
sentence = "It costs 12.21$, 80% discount"

for tokenizer in [MyTokenizer(), WordTokenizer(), NLTKTokenizer()]:
    print(tokenizer)
    print(tokenizer.tokenize(sentence))

MyTokenizer
['It', 'costs', '12', '21', '80', 'discount']
WordTokenizer
['It', 'costs', '12.21', '$', ',', '80', '%', 'discount']
NLTKTokenizer
['It', 'costs', '12', '.', '21', '$,', '80', '%', 'discount']


In [ ]:
for tokenizer in [MyTokenizer(), WordTokenizer(), NLTKTokenizer()]:
    print(tokenizer)
    validate(validation_data, wv_embeddings, tokenizer)

MyTokenizer


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.524 | Hits@  10: 0.650
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000
WordTokenizer


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.399 | Hits@   1: 0.399
DCG@   5: 0.487 | Hits@   5: 0.566
DCG@  10: 0.509 | Hits@  10: 0.634
DCG@ 100: 0.554 | Hits@ 100: 0.858
DCG@ 500: 0.569 | Hits@ 500: 0.969
DCG@1000: 0.573 | Hits@1000: 1.000
NLTKTokenizer


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.410 | Hits@   1: 0.410
DCG@   5: 0.500 | Hits@   5: 0.580
DCG@  10: 0.521 | Hits@  10: 0.644
DCG@ 100: 0.568 | Hits@ 100: 0.875
DCG@ 500: 0.580 | Hits@ 500: 0.973
DCG@1000: 0.583 | Hits@1000: 1.000


**Вывод:** Для предобученных эмбеддингов лучше себя показал наивный токенизатор, который игнорирует знаки препинания и прочие символы, отличные от цифр и букв.  

### Добавление нормализации (стемминга)

**Замечание:** Так как слова в предобученных эмбеддингах не были нормализованы (есть слова `cat` и `cats`), то этот эксперимент будет проводиться на обучаемых эмбеддингах

In [ ]:
from nltk.stem import SnowballStemmer


stemmer = SnowballStemmer(language="english")
tokenizer = MyTokenizer()

norm_train_data = read_corpus('./data/train.tsv', norm_func=stemmer.stem)
norm_val_data = read_corpus('./data/validation.tsv', norm_func=stemmer.stem)

norm_train_words = get_train_words(norm_train_data, tokenizer)
norm_embeddings = Word2Vec(train_words,
                           size=200,
                           min_count=5,
                           window=3).wv

print("Before normalization")
validate(validation_data, embeddings_trained, tokenizer)

print("After normalization")
validate(norm_val_data, norm_embeddings, tokenizer)

Before normalization


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.291 | Hits@   1: 0.291
DCG@   5: 0.369 | Hits@   5: 0.438
DCG@  10: 0.386 | Hits@  10: 0.493
DCG@ 100: 0.442 | Hits@ 100: 0.768
DCG@ 500: 0.464 | Hits@ 500: 0.936
DCG@1000: 0.471 | Hits@1000: 1.000
After normalization


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.296 | Hits@   1: 0.296
DCG@   5: 0.369 | Hits@   5: 0.432
DCG@  10: 0.390 | Hits@  10: 0.500
DCG@ 100: 0.445 | Hits@ 100: 0.767
DCG@ 500: 0.467 | Hits@ 500: 0.940
DCG@1000: 0.474 | Hits@1000: 1.000


**Вывод:** Нормализация слов незначительно улучшила качество арнжирования

### Фильтрация токенов для обучения эмбеддингов

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords


stopWords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.corpus import stopwords


def filter_word(word):
    if len(word) <= 1 or word in stopWords:
        return False

    return True

filtered_norm_words = []
tokenizer = MyTokenizer()

for words_list in norm_train_words:
    tmp_list = list(filter(filter_word, words_list))
    filtered_norm_words.append(tmp_list)

filtered_norm_embeddings = Word2Vec(filtered_norm_words,
                                    size=200,
                                    min_count=5,
                                    window=3).wv
validate(norm_val_data, filtered_norm_embeddings, tokenizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.407 | Hits@   1: 0.407
DCG@   5: 0.497 | Hits@   5: 0.578
DCG@  10: 0.524 | Hits@  10: 0.659
DCG@ 100: 0.568 | Hits@ 100: 0.873
DCG@ 500: 0.581 | Hits@ 500: 0.977
DCG@1000: 0.584 | Hits@1000: 1.000


Качество ранжирования для предобученных эмбеддингов:  
DCG@   1: 0.415 | Hits@   1: 0.415 \\
DCG@   5: 0.502 | Hits@   5: 0.582 \\
DCG@  10: 0.524 | Hits@  10: 0.650 \\
DCG@ 100: 0.570 | Hits@ 100: 0.874 \\
DCG@ 500: 0.583 | Hits@ 500: 0.973 \\
DCG@1000: 0.586 | Hits@1000: 1.000 \\

**Вывод:** Как можно видеть, оба способа получения эмбеддингов имеют приблизительно одинаковое качество

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:
